## Data Cleaning and Preprocessing Notebook

#### Dataset Description:
##### Medication Inventory Management Dataset - Michigan Medicine
The dataset contains historical medication inventory and transaction data from Michigan Medicine, covering 3 hospitals, 40 outpatient locations, and over 120 clinics. It includes detailed information on inventory levels, transactions (picks, restocks, wastes, etc.), medication details, locations, and costs.

|                  |                                                     |
|--------------------------|---------------------------------------------------------------|
| **Estimated size**       | 3,807,314 rows and multiple columns (approximately 1GB)       |
| **Format**               | Comma Separated Values (.csv)                                 |
| **Location & Access**    | Access provided by the Pharmacy Manager (Michigan Medicine - Pharmacy Administration) |

##### More About the Variables
| Column Name             | Description                                                                                             |
|:------------------------|:--------------------------------------------------------------------------------------------------------|
| **daily_inv_location**  | The physical location where inventory was counted on a given day.                                       |
| **daily_inv_date**      | The date of the inventory count.                                                                        |
| **isa_name**            | The name of the inventory storage area.                                                                 |
| **daily_inv_med_id**    | A unique identifier for the medication counted during inventory.                                        |
| **med_id_clean**        | A standardized version of the medication identifier. Often combines multiple med_id that are suffixed into one unit. |
| **med_description**     | A brief description of the medication.                                                                  |
| **first_count_of_day**  | The initial count of the medication on the inventory date.                                              |
| **last_count_of_day**   | The final count of the medication on the inventory date.                                                |
| **next_daily_inv_date** | The date of the next transaction (and therefore inventory change).                                      |
| **calendar_dt**         | A specific calendar date for reference. Should always be present regardless of whether there is a transaction for the med on a given day. |
| **pick**                | The quantity of medication picked from inventory.                                                       |
| **cycle_count**         | The quantity of medication cycle counted.                                                               |
| **waste**               | The quantity of medication wasted or discarded.                                                         |
| **destock**             | The quantity of medication removed from active inventory.                                               |
| **batch_pick**          | The quantity of medication picked in a batch process.                                                   |
| **load**                | The quantity of medication loaded into a dispensing system.                                             |
| **inventory**           | The total quantity of medication processed using the inventory function.                                |
| **restock**             | The quantity of medication added back into inventory.                                                   |
| **current_inv_med_id**  | The current unique identifier for the medication in inventory.                                          |
| **current_inv_location**| The current location of the medication in inventory.                                                    |
| **current_inv_min**     | The minimum quantity of the medication to be kept in inventory. Inventory below this value will create an order for restock. |
| **current_inv_max**     | The maximum quantity of the medication to be kept in inventory. This is the order up to level.          |
| **current_inv_qoh**     | The current quantity on hand (QOH) of the medication.                                                   |
| **pref_ndc_vendor_name**| The preferred vendor's name for the medication's National Drug Code (NDC).                              |
| **pref_ndc_package_size**| The preferred package size associated with the medication's NDC.                                       |
| **pref_ndc**            | The preferred National Drug Code (NDC) for the medication.                                              |
| **scd_name**            | The Standardized Concept Drug (SCD) name from RxNorm.                                                   |
| **scd_rxcui**           | The RxNorm Concept Unique Identifier (RxCUI) for the SCD.                                               |
| **in_min_name**         | The Ingredient Minimum (IN) name from RxNorm.                                                           |
| **in_min_rxcui**        | The RxNorm Concept Unique Identifier (RxCUI) for the IN.                                                |
| **source_description**  | A description of the med from the source system.                                                        |


##### Key Variables
|                          |                                                                                                         |
|:-------------------------|:--------------------------------------------------------------------------------------------------------|
| **daily_inv_location**   | This will be important for location-based analysis and predictions                                      |
| **daily_inv_date**       | This will be crucial for time-based analysis and seasonality detection                                  |
| **med_id_clean**         | Standardized medication identifier, it will be crucial for consistent analysis across different medication types |
| **pick**                 | Best surrogate for what left the inventory system, this will be key for demand forecasting              |
| **waste**                | This can be important for identifying medications prone to waste and optimizing inventory              |
| **restock**              | This will be important for inventory management as it indicates what goes back into the system          |
| **current_inv_min and current_inv_max** | This will be important for understanding current inventory management strategies         |
| **current_inv_qoh**      | Current quantity on hand, it will be essential for inventory level optimization                         |

##### Key Considerations

|                  |                                                     |
|--------------------------|---------------------------------------------------------------|
| **Geographic Scope:** | Single health system in Michigan, with a central pharmacy distributing to multiple locations       |
| **Business Objective:** | Reduce waste while minimizing stockouts, with very low tolerance for stockouts       |
| **Key Metric:** | Days on hand (DoH), with a general target of a couple of week       |
| **Potential Factors:** | Spikes in inventory followed by long drifts down may indicate bulk buys due to anticipated shortages |
| **Stockouts** | can be caused by shortages, but we don't have data on shortage timelines                                 |

**Note:** Days on Hand (DoH) estimates how long current inventory will last based on historical or projected usage rates [1]. In our case, it can represent how long the current stock of a medication would last based on expected usage. For example, if the pharmacy has 300 units of a certain medication and uses 10 units per day, then it has a 30 day supply on hand.

A low DoH based on historical usage may indicate a potential risk of running out of a medication soon. On the other hand, a high DoH might indicate excessive inventory which can lead to expired medications.

#### Setup and Data Loading:
##### Import Libraries
To start data cleaning and manipulation, we are going to import and install some helper libraries that we will need to process the
 data.

In [22]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis

pd.set_option('display.max_columns', None)

##### Load Dataset
Next, we are going to load the dataset and examine its structure such as shape, column names, data types and so on.

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# data = pd.read_csv('pharmacy_central_inventory_and_transactions_by_date.csv')
# Data path for running in collab with mounted Drive
raw_data = pd.read_csv('/content/drive/MyDrive/CAPSTONE/pharmacy_central_inventory_and_transactions_by_date.csv')

<ipython-input-24-b4d6cf4a9cec>:3: DtypeWarning: Columns (2,3,4,18,19,24,27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('/content/drive/MyDrive/CAPSTONE/pharmacy_central_inventory_and_transactions_by_date.csv')


In [25]:
data = raw_data.copy()
print(f"Dataset shape: {data.shape}")
print("First few rows of the dataset:")
data.head(5)

Dataset shape: (3807314, 32)
First few rows of the dataset:


,daily_inv_location,daily_inv_date,isa_name,daily_inv_med_id,med_id_clean,med_description,first_count_of_day,last_count_of_day,next_daily_inv_date,calendar_dt,pick,cycle_count,waste,destock,batch_pick,load,inventory,restock,current_inv_med_id,current_inv_location,current_inv_min,current_inv_max,current_inv_qoh,reviewed_number,pref_ndc_vendor_name,pref_ndc_package_size,pref_ndc,scd_name,scd_rxcui,in_min_name,in_min_rxcui,source_description
0,2C224-01-01-01-01,2022-03-14,NaN,001185,001185,sodium bicarbonate 8.4 % solution - 50 mL vial,21909.0,22909.0,2022-03-15,2022-03-14 00:00:00.000,-25.0,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,3.22,AB Short A,25.0,409662514.0,NaN,NaN,NaN,NaN,NaN
1,2C224-01-01-01-01,2022-03-15,NaN,001185,001185,sodium bicarbonate 8.4 % solution - 50 mL vial,22909.0,22509.0,2022-03-16,2022-03-15 00:00:00.000,-424.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.22,AB Short A,25.0,409662514.0,NaN,NaN,NaN,NaN,NaN
2,2C224-01-01-01-01,2022-03-16,NaN,001185,001185,sodium bicarbonate 8.4 % solution - 50 mL vial,22509.0,22509.0,2022-03-17,2022-03-16 00:00:00.000,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.22,AB Short A,25.0,409662514.0,NaN,NaN,NaN,NaN,NaN
3,2C224-01-01-01-01,2022-03-17,NaN,001185,001185,sodium bicarbonate 8.4 % solution - 50 mL vial,22509.0,22009.0,2022-03-20,2022-03-17 00:00:00.000,-618.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.22,AB Short A,25.0,409662514.0,NaN,NaN,NaN,NaN,NaN
4,2C224-01-01-01-01,2022-03-17,NaN,001185,001185,sodium bicarbonate 8.4 % solution - 50 mL vial,22009.0,22009.0,2022-03-20,2022-03-18 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.22,AB Short A,25.0,409662514.0,NaN,NaN,NaN,NaN,NaN


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3807314 entries, 0 to 3807313
Data columns (total 32 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   daily_inv_location     object 
 1   daily_inv_date         object 
 2   isa_name               object 
 3   daily_inv_med_id       object 
 4   med_id_clean           object 
 5   med_description        object 
 6   first_count_of_day     float64
 7   last_count_of_day      float64
 8   next_daily_inv_date    object 
 9   calendar_dt            object 
 10  pick                   float64
 11  cycle_count            float64
 12  waste                  float64
 13  destock                float64
 14  batch_pick             float64
 15  load                   float64
 16  inventory              float64
 17  restock                float64
 18  current_inv_med_id     object 
 19  current_inv_location   object 
 20  current_inv_min        float64
 21  current_inv_max        float64
 22  current_inv_qoh   

#### Data Cleaning and Preprocessing:
##### Check for Duplicates and Identify Missing Values
Next, we will check for duplicate rows to ensure that each row is unique and avoid any redundancy in the dataset.

In [27]:
# Check for duplicate records
print(f"Number of duplicate records: {data.duplicated().sum()}")

Number of duplicate records: 0


In [28]:
# Check for missing values
data.isnull().sum()

,0
daily_inv_location,0
daily_inv_date,0
isa_name,1344638
daily_inv_med_id,0
med_id_clean,23009
med_description,23009
first_count_of_day,4450
last_count_of_day,3658
next_daily_inv_date,0
calendar_dt,0


##### Drop irrelevant medications
We'll next identify:


1.   `daily_inv_med_id` values that have 0 total picks across the dataset
2.   `daily_inv_med_id` values that have had no inventory entires for the past 120 days.

Any `daily_inv_med_id` values meeting either of the above criteria will be dropped from the datatset.

In [29]:
# TODO - I think all of this should actually get moved to after we do the aggregations
# Initialize set to track daily_inv_med_id values to drop
#med_ids_to_drop = set()

In [30]:
# Identify daily_inv_med_id values with 0 picks across the entirety of the data
#total_picks = data.groupby('daily_inv_med_id')['pick'].sum()
#no_picks = total_picks[total_picks == 0].index

#print(f"{len(no_picks)} daily_inv_med_id values with no picks across the dataset.")

In [31]:
# Identify daily_inv_med_id values that haven't had inventory entries for the past 120 days
# TODO - perhaps we change this to be like meds missing > 50% of days of data
#day_cutoff = pd.to_datetime(data['calendar_dt'].max()) - pd.Timedelta(days=120)
#max_inv_date = data.groupby('daily_inv_med_id')['calendar_dt'].max()
#below_cutoff = max_inv_date[pd.to_datetime(max_inv_date) < day_cutoff].index

#print(f"{len(below_cutoff)} daily_inv_med_id values with no inventory records in the past 120 days.")

In [32]:
# Iterate through each criteria; iterate through each daily_inv_med_id in the criteria;
# then add the daily_inv_med_id values to our set of daily_inv_med_ids to drop
#for criteria in [no_picks, below_cutoff]:
#  for med_id in criteria:
#    med_ids_to_drop.add(med_id)

# Drop irrelevant med_ids from the dataset
#data = data[~data['daily_inv_med_id'].isin(med_ids_to_drop)]

#print(f"Dropped {len(med_ids_to_drop)} daily_inv_med_id values from dataset")

In [33]:
# Note: we may want to consider some other criteria, such as:
# - meds without a pick for the past 180 days
# - meds with a current_inv_max value of 0 (which may insinuate they only purchase on an ad hoc basis) -- but maybe this is just a bug and
# we fix through the code I have below?
# - meds that average less than .5 picks per date (b/c if we forecast using integers, this would always round down to 0)

#### Fill in Missing or Inconsistent Inventory Min./Max. Levels

For every medication (`daily_inv_med_id`) there should be a specified inventory minimum (`current_inv_min`) and maximum (`current_inv_max`). A reorder will automatically be triggered upon a mediciation's quantitiy on hand reaching the minimum threshold. By default, the quantitiy reordered will be whatever volume brings the quantitiy on hand back to the maximum threshold. However, this can be overriden by whomever is placing the order.

While a mediciation's min./max. values can fluctuate over time (albeit, not often), the values within the dataset reflect the values at the time at which the data was queried (i.e., historical min./max. values are not maintained)

We'll use the following logic to fill in missing values.


*   If a medication only has one historic min./max. value, that value will be used to backfill missing values.
*   If a mediciation has multiple historic min./max. values, the most frequent min./max. values will be used to backfill missing values.
*   If a medication has no min./max. values:
  * We'll use the 5th percentile of all `last_count_of_day` values to estimate the inventory minimum.
  *   We'll use the 95th percentile of all `first_count_of_day` values to estimate the inventory maximum.

This may beg the question, “Why use the 5th percentile as an estimate for the inventory minimum and not the absolute minimum?” We chose to do so because we would expect the lowest last_count_of_day values to denote the days when a restock was delayed, thus sinking the inventory volume beneath the true inventory minimum. With this in mind, we opted to use the 5th percentile instead of the 0th percentile (i.e., the absolute minimum) to ensure we did not underestimate the true inventory minimum.

Similarly, we chose to use the 95th percentile of first_count_of_day values to estimate the maximum inventory level. We did so because we would expect the highest first_count_of_day values to denote the days when inventory was intentionally over-stocked, thus pushing the inventory volume above the true inventory minimum. With this in mind, we opted to use the 95th percentile instead of the 100th percentile (i.e., the absolute maximum) to ensure we did not overestimate the true inventory maximum.

In [34]:
# We'll then create a grouped df of all medications which tracks the following for both the min and max inventory levels:
# - Count of null values
# - Count of unique values
# - Most frequent value
# - 5th percentile of the last_count_of_day / 95th percentile of the first_count_of_day

min_max_ref = data.groupby('daily_inv_med_id').agg(
    min_nulls=('current_inv_min', lambda x: x.isna().sum()), # Number of null values
    min_unique=('current_inv_min', lambda x: x.nunique(dropna=True)), # Number of unique values excluding nulls
    min_most_freq=('current_inv_min', lambda x: x.mode().iloc[0] if not x.mode().empty else None), # Most frequent value
    last_ct_percentile=('last_count_of_day', lambda x: x.quantile(0.05)), # 5th percentile

    max_nulls=('current_inv_max', lambda x: x.isna().sum()), # Number of null values
    max_unique=('current_inv_max', lambda x: x.nunique(dropna=True)), # Number of unique values excluding nulls
    max_most_freq=('current_inv_max', lambda x: x.mode().iloc[0] if not x.mode().empty else None), # Most frequent value
    first_ct_percentile=('first_count_of_day', lambda x: x.quantile(0.95)) # 95th percentile
)

# Create new columns to store our "clean" values for the inventory min/max values
min_max_ref[['clean_current_inv_min', 'clean_current_inv_max']] = None, None

# Apply business rules
min_max_ref['clean_current_inv_min'] = min_max_ref.apply(lambda row: row['min_most_freq'] if pd.notna(row['min_most_freq']) else row['last_ct_percentile'], axis=1)
min_max_ref['clean_current_inv_max'] = min_max_ref.apply(lambda row: row['max_most_freq'] if pd.notna(row['max_most_freq']) else row['first_ct_percentile'], axis=1)

# View a sample of the dataframe we created to check the results
min_max_ref.sample(5)

,min_nulls,min_unique,min_most_freq,last_ct_percentile,max_nulls,max_unique,max_most_freq,first_ct_percentile,clean_current_inv_min,clean_current_inv_max
daily_inv_med_id,,,,,,,,,,
P3510,672,0,NaN,0.0,672,0,NaN,0.0,0.0,0.0
P4189vc1,0,1,0.0,0.0,0,1,0.0,0.0,0.0,0.0
852898,0,1,10.0,8.0,0,1,30.0,63.0,10.0,30.0
028090P02,505,0,NaN,0.0,505,0,NaN,4.0,0.0,4.0
029916,0,1,2.0,1.0,0,1,2.0,3.0,2.0,2.0


In [35]:
# Now let's merge in the "clean" min / max values back to the main df
data = pd.merge(data, min_max_ref[['clean_current_inv_min', 'clean_current_inv_max']], left_on='daily_inv_med_id', right_index=True)

##### Standardize Medication IDs

Next, we will take a closer look at the missing values for 'med_id_clean' since it will be crucial for consistent analysis across different medication types.

In [36]:
# Check identical and different values for 'daily_inv_med_id' and 'med_id_clean' for each row
identical_ids = data['daily_inv_med_id'] == data['med_id_clean']
different_ids = data['daily_inv_med_id'] != data['med_id_clean']

# Count how many are identical and how many are different
identical_count = identical_ids.sum()
different_count = different_ids.sum()

print(f"Number of rows where 'daily_inv_med_id' and 'med_id_clean' are identical: {identical_count}")
print(f"Number of rows where 'daily_inv_med_id' and 'med_id_clean' are different: {different_count}")

Number of rows where 'daily_inv_med_id' and 'med_id_clean' are identical: 3133789
Number of rows where 'daily_inv_med_id' and 'med_id_clean' are different: 673525


Overall majority of ids in both 'daily_inv_med_id' and 'med_id_clean' seems to be identical with approximately 673,525 different ids. Next, we will check for any inconsistencies and try to observe some common patterns between the two.

In [37]:
identical_rows = data[identical_ids]
print("Rows where 'daily_inv_med_id' and 'med_id_clean' are identical:")
print(identical_rows[['daily_inv_med_id', 'med_id_clean']].sample(10))

Rows where 'daily_inv_med_id' and 'med_id_clean' are identical:
            daily_inv_med_id         med_id_clean
3394659               061119               061119
2765080               022678               022678
3503327          TEMP072122Z          TEMP072122Z
2546928               081250               081250
1329126               023881               023881
2057994               046103               046103
3325565               080567               080567
270184                045191               045191
2065437               039138               039138
3688858  P3798:  HUM00213019  P3798:  HUM00213019


In [38]:
different_rows = data[different_ids]
print("Rows where 'daily_inv_med_id' and 'med_id_clean' are different:")
print(different_rows[['daily_inv_med_id', 'med_id_clean']].sample(20))

Rows where 'daily_inv_med_id' and 'med_id_clean' are different:
        daily_inv_med_id med_id_clean
3001840          071277        071277
2248943           076360        76360
2330394           075111        75111
2311734           047874        47874
645518            006748         6748
1156530           006787         6787
1935247       005124bulk       005124
2095384       008222bulk       008222
1895986       029161bulk        29161
633872            042944        42944
1708468           804581       804581
2043926       046047bulk       046047
2123896       024668bulk        24668
1895784       029161bulk        29161
968693            011640      11640.0
2317529           009542         9542
2145743       065328bulk       065328
154696        004494bulk       004494
1790010           017134        17134
1782722       002214bulk       002214


Although the 'med_id_clean' column appears to be standardized medication identifier, there are some inconsistencies such as: i) how leading zeros are handled (most of the time it is kept, but sometimes removed); ii) data type inconsistencies, with some values as floats but majority as string.

We will standardize 673,525 different ids to the common patterns observed to create more consistent 'med_id_clean' values such as keeping leading zeros, removal of suffix (bulk) to keep numeric values only and conversion of floats.

In [39]:
# Create a new 'cleaned_med_id' column, initialize it with 'med_id_clean' values
data['cleaned_med_id'] = data['med_id_clean']

# Apply the cleaning operation only to rows that are different
data.loc[different_ids, 'cleaned_med_id'] = data.loc[different_ids, 'daily_inv_med_id'].apply(
    lambda x: re.sub(r'bulk$', '', str(x), flags=re.IGNORECASE).strip())

In [40]:
# Let's confirm
different_rows = data[different_ids]
print("Rows where 'daily_inv_med_id' and 'cleaned_med_id' are different:")
print(different_rows[['daily_inv_med_id', 'cleaned_med_id', 'med_id_clean']].sample(10))

Rows where 'daily_inv_med_id' and 'cleaned_med_id' are different:
        daily_inv_med_id cleaned_med_id med_id_clean
281757        048044bulk         048044       048044
1858155       046068bulk         046068       046068
1982931       000347bulk         000347          347
1977214       046238bulk         046238       046238
1883927           040910         040910        40910
2260518           060046         060046        60046
2053742       016600bulk         016600       016600
2103355       000267bulk         000267          267
392349        012016bulk         012016       012016
1940315       046214bulk         046214       046214


In [41]:
data[['daily_inv_location', 'daily_inv_med_id', 'med_id_clean', 'cleaned_med_id', 'pick', 'waste', 'restock',
      'current_inv_min',  'current_inv_max', 'current_inv_qoh']].isna().sum()

,0
daily_inv_location,0
daily_inv_med_id,0
med_id_clean,23009
cleaned_med_id,0
pick,2889223
waste,3805317
restock,3494103
current_inv_min,1344638
current_inv_max,1344638
current_inv_qoh,1344638


After standardizing medication IDs, missing values seems to be resolved, this looks much better.

#### Data Aggregation:
For other important numerical columns, we will first aggregate the data to weekly level. Doing this will reduce the granularity of our time series and have a smoothing effect to reduce the impact of daily fluctuations or missing values. In other words, this will help us handle missing values more gracefully as well as reduce the sparsity of the data.

Given that it's a single health system with a central pharmacy distributing to multiple locations, it seems appropriate to aggregate across locations. This approach would give us a system-wide view of inventory for each medication.

In [42]:
# Convert date columns to datetime type
date_columns = ['daily_inv_date', 'next_daily_inv_date', 'calendar_dt']
data[date_columns] = data[date_columns].apply(pd.to_datetime)

# Convert 'pick' and 'waste' to absolute values before aggregation
data['abs_pick'] = data['pick'].abs()
data['abs_waste'] = data['waste'].abs()

We'll now export a copy of the cleaned timeseries data. This will be used locally for exploratory data analysis using Tableau.

In [43]:
#data.to_csv('/content/drive/MyDrive/CAPSTONE/cleaned_inventory_data.csv', index=False)

Up to this point, the data is disaggregated by:
1.   Individual calendar days
2.   Location (although those locations may be in very close proximity to one another, e.g., an adjacent storage room)
3.   What size container they're stored in (e.g., a container of 500 pills vs. a storage container with a small quantity of individual pills).

Our goal is to create an aggregated dataframe where each observation represents one day of inventory data for a unique medication by dosage amount and dosage form. To achieve  this we'll group the data by the `cleaned_med_id` value and perform aggregations on the key features of interest.

**Limitation**: a critical limitation to note with this approach is that pick values may represent medications being transferred from a bulk container to a non-bulk container. For example, if 500 pills for a given medication are picked from a bulk container and distributed across various locations, the data would view this as a pick, whereas in reality this isn't *really* a pick; it's a redistribution of inventory. With how we received the data, we are unable to differentiate these instances of picks vs. picks that entail medications *actually* leaving the inventory system to be given to end-users. Because of this, all `cleaned_med_id` values with both a bulk and non-bulk `daily_inv_med_id` derivative may be susceptible to double-counting picks.

In [44]:
df_agg = data.groupby(['cleaned_med_id', 'calendar_dt']).agg({
    'abs_pick': 'sum',
    'clean_current_inv_min': 'sum',
    'clean_current_inv_max': 'sum',
    'first_count_of_day': 'sum',
    'last_count_of_day': 'sum'
}).reset_index()

df_agg.columns = ['cleaned_med_id', 'calendar_dt', 'pick', 'clean_inv_min',
                  'clean_inv_max', 'first_count_of_day', 'last_count_of_day']

df_agg.head(5)

,cleaned_med_id,calendar_dt,pick,clean_inv_min,clean_inv_max,first_count_of_day,last_count_of_day
0,231,2022-03-14,5.0,0.0,48.0,0.0,25.0
1,231,2022-03-15,0.0,0.0,48.0,25.0,25.0
2,231,2022-03-16,0.0,0.0,48.0,25.0,25.0
3,231,2022-03-17,0.0,0.0,48.0,25.0,25.0
4,231,2022-03-18,0.0,0.0,48.0,25.0,25.0


We'll now create a new dataframe that further aggregates the grouped `cleaned_med_id` time-series data so that we have one row per unique medication. For each medication value we'll track:

1.   Minimum inventory level
2.   Maximum inventory level
3.   Mean first count of day
4.   Mean last count of day
5.   Total picks
6.   Mean picks per day
7.   Median picks per day
8.   Variance of picks per day
9.   Standard deviation of picks per day
10.   Range of picks per day
11.   Interquartile range of picks per day
12.   Skewness of picks per day
13.   Kurtosis of picks per day
14.   Percent of days with non-zero picks
15.   Number of days with data present (even if there was no movement)
16.   Number of days with a stockout
17.   Number of days with scarce inventory (defined as `last_count_of_day` < `minimum_inventory_level`)
18.   Number of days with excess inventory (defined as `last_count_of_day` > `maximum_inventory_level`)

In [45]:
def calculate_statistics(group):
    pick_count = group['pick'].notnull() & (group['pick'] != 0)
    stockout = group['last_count_of_day'] == 0
    below_min = group['last_count_of_day'] < group['clean_inv_min']
    above_max = group['last_count_of_day'] > group['clean_inv_max']

    return pd.Series({
        'inv_min': group['clean_inv_min'].mean(),
        'inv_max': group['clean_inv_max'].mean(),
        'mean_first_count': group['first_count_of_day'].mean(),
        'mean_last_count': group['last_count_of_day'].mean(),
        'total_picks': group['pick'].sum(),
        'mean_picks': group['pick'].mean(),
        'median_picks': group['pick'].median(),
        'variance_picks': group['pick'].var(),
        'std_dev_picks': group['pick'].std(),
        'range_picks': group['pick'].max() - group['pick'].min(),
        'iqr_picks': group['pick'].quantile(0.75) - group['pick'].quantile(0.25),
        'skewness_picks': skew(group['pick']),
        'kurtosis_picks': kurtosis(group['pick']),
        'percent_days_w_pick': pick_count.sum() / group['calendar_dt'].count(),
        'days_of_data': group['calendar_dt'].count(),
        'days_w_stockout': stockout.sum(),
        'days_w_scarce_inv': below_min.sum(),
        'days_w_excess_inv': above_max.sum(),
    })

In [46]:
# Iterate through each medication and execute the calculate_statistics function
summary_df = df_agg.groupby('cleaned_med_id').apply(calculate_statistics).reset_index()

With the aggregate statistics now available, we'll engineer some additional features:


*   **Mean Inventory** - Measures the average volume on hand across the period of data.
*   **Inventory Turnover Ratio** - Measures the number of times the average volume on hand is exhausted over the period of data. A higher ratio implies efficient inventory management. A lower ratio denotes overstocking.
*   **Days Sales of Inventory (DSI)** - Measures the average number of days it takes to exhaust the average inventory level. A Lower DSI indicates quick inventory turnover and efficient inventory management. A higher DSI suggests slower inventory turnover.
*   **Stockout, Scarce Inventory, and Excess Inventory Rates** -  Measures the percentage of days whether there respectively was a stockout, scarce inventory, or excess inventory.
*   **Coefficient of Variation** -  Normalizes standard deviation values by the mean volume of picks.

In [47]:
summary_df['mean_inventory'] = summary_df.apply(lambda x: ((x.mean_first_count + x.mean_last_count) / 2), axis = 1)
summary_df['inv_turnover_ratio'] = summary_df.apply(lambda x: x.total_picks / x.mean_inventory if x.mean_inventory != 0 else np.nan, axis = 1)
summary_df['days_sales_inventory'] = summary_df.apply(lambda x: x.mean_inventory / x.mean_picks if x.mean_picks != 0 else np.nan, axis = 1)
summary_df['stockout_rate'] = summary_df.apply(lambda x: x.days_w_stockout / x.days_of_data, axis = 1)
summary_df['excess_rate'] = summary_df.apply(lambda x: x.days_w_excess_inv / x.days_of_data, axis = 1)
summary_df['scarce_rate'] = summary_df.apply(lambda x: x.days_w_scarce_inv / x.days_of_data, axis = 1)
summary_df['coef_var'] = summary_df.apply(lambda x: x.std_dev_picks / x.mean_picks if x.mean_picks != 0 else np.nan, axis = 1)

summary_df.set_index('cleaned_med_id', drop = True, inplace = True)
summary_df.describe()

,inv_min,inv_max,mean_first_count,mean_last_count,total_picks,mean_picks,median_picks,variance_picks,std_dev_picks,range_picks,iqr_picks,skewness_picks,kurtosis_picks,percent_days_w_pick,days_of_data,days_w_stockout,days_w_scarce_inv,days_w_excess_inv,mean_inventory,inv_turnover_ratio,days_sales_inventory,stockout_rate,excess_rate,scarce_rate,coef_var
count,4593.000000,4593.000000,4593.000000,4593.000000,4.593000e+03,4593.000000,4593.000000,4.592000e+03,4592.000000,4593.000000,4593.000000,3795.000000,3795.000000,4593.000000,4593.000000,4593.000000,4593.000000,4593.000000,4593.000000,3.915000e+03,3.795000e+03,4593.000000,4593.000000,4593.000000,3795.000000
mean,108.462499,207.126687,545.266230,544.660901,1.262575e+04,15.778952,5.194426,1.902345e+06,152.162864,3987.915306,8.745972,9.095319,149.107123,0.243645,683.247115,119.734814,109.490529,190.256260,544.963566,6.299139e+02,1.396594e+03,0.259409,0.251089,0.140845,6.604906
std,910.460055,1417.151325,8648.447441,8639.612309,8.156108e+04,100.288268,34.468183,3.050799e+07,1370.985439,38605.305442,103.707385,7.328474,210.110974,0.316009,230.915142,210.649000,179.552887,223.760386,8644.025455,1.897880e+04,4.428277e+04,0.401275,0.291911,0.227393,6.777938
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-0.117382,-1.417104,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.195339
25%,0.000000,0.000000,2.150062,1.982895,8.000000e+00,0.013819,0.000000,2.616510e-02,0.161756,2.000000,0.000000,3.669349,16.425501,0.006258,676.000000,1.000000,0.000000,2.000000,2.034314,4.642522e+00,1.340504e+01,0.001218,0.003645,0.000000,2.061300
50%,6.000000,17.000000,21.212766,21.064304,2.430000e+02,0.338200,0.000000,2.261757e+00,1.503914,18.000000,0.000000,6.626117,55.946768,0.084158,812.000000,8.000000,18.000000,98.000000,21.077491,1.898519e+01,3.262819e+01,0.010936,0.128797,0.024450,3.817028
75%,40.000000,111.000000,101.309002,101.209246,2.364000e+03,3.179830,0.000000,5.603201e+01,7.485452,83.000000,2.000000,12.503739,180.254633,0.388480,822.000000,140.000000,141.000000,315.000000,101.290754,5.001896e+01,1.022619e+02,0.474201,0.411192,0.181929,8.533286
max,32000.000000,50000.000000,364342.823815,364291.623329,2.813873e+06,3423.203163,1531.000000,1.117813e+09,33433.706056,959166.000000,5315.000000,28.635663,818.001217,1.000000,823.000000,823.000000,823.000000,823.000000,364317.223572,1.141090e+06,2.631315e+06,1.000000,1.000000,1.000000,28.687977


We'll now export a copy of the data frame containing the aggregate summary statistics. This will be used for local exploratory data analysis using Tableau and subsequent machine learning analyses.

In [48]:
# Write file to CAPSTONE Google Drive - used to download + do more EDA in Tableau locally
summary_df.to_csv('/content/drive/MyDrive/CAPSTONE/inventory_summary_statistics_stage1.csv', index=True)

In [ ]:
# Aggregate data to weekly level
data['week'] = data['calendar_dt'].dt.to_period('W').dt.start_time
weekly_data = data.groupby(['week', 'cleaned_med_id']).agg({
    'abs_pick': 'sum',
    'abs_waste': 'sum',
    'restock': 'sum',
    'current_inv_min': 'sum',
    'current_inv_max': 'sum',
    'current_inv_qoh': 'sum',
    'daily_inv_location': ['nunique', lambda x: list(x.unique())]
}).reset_index()

weekly_data.columns = ['week', 'cleaned_med_id', 'pick', 'waste', 'restock', 'inv_min', 'inv_max', 'inv_qoh', 'location_count', 'location_id']

# Rename the columns for consistency
# weekly_data = weekly_data.rename(columns={'abs_pick': 'pick', 'abs_waste': 'waste', 'daily_inv_location': 'location_count'})

In [ ]:
weekly_data.head()

,week,cleaned_med_id,pick,waste,restock,inv_min,inv_max,inv_qoh,location_count,location_id
0,2022-03-07,4688,0.0,0.0,1.0,100.0,150.0,229.0,1,[Car3-21-02-02]
1,2022-03-07,4763,1.0,0.0,0.0,0.0,0.0,0.0,1,[Car3-21-08-02]
2,2022-03-07,45017,0.0,0.0,1.0,0.0,0.0,0.0,1,[Car3-21-08-03]
3,2022-03-07,69921,0.0,0.0,1.0,2.0,4.0,2.0,1,[Car1-31-12-01]
4,2022-03-07,69928,0.0,0.0,19.0,0.0,0.0,74.0,1,[Car1-31-12-03]


This should gives us system-wide totals for pick, waste, restock, minimum and maximum inventory and current inventory on hand for each medication per week.

#### Feature Engineering:
##### Calculate Expected Usage (4 week moving average)
Next, we will focus on the days on hand (DoH) metric but first we need to calculate the expected usage.

Since we are working with weekly data, we will calculate expected usage using 4 week moving average of historical usage. It basically represents how much of the medication we expect to use in a week based on the average usage over the last 4 weeks. So we will average 'pick' over the last 4 weeks for each medication at each location. This will give us about a month of historical use which will not be overly sensitive to single week spikes or drops.

In [ ]:
# Sort the data
weekly_data = weekly_data.sort_values(['cleaned_med_id', 'week'])

# Calculate 4 week moving average for expected usage
weekly_data['expected_usage'] = weekly_data.groupby(['cleaned_med_id'])['pick'].transform(
    lambda x: x.rolling(window=4, min_periods=1).mean())

##### Calculate Days on Hand (DoH)
Now we can calculate days on hand based on the current inventory and expected usage. So basically this will give us the number of days the current inventory would last based on the average usage over the last 4 weeks.

In [ ]:
# Calculate 'days on hand' metric using expected usage
weekly_data['days_on_hand'] = np.where((weekly_data['inv_qoh'] >= 0) & (weekly_data['expected_usage'] > 0),
                                        weekly_data['inv_qoh'] / (weekly_data['expected_usage'] / 7), np.nan)

##### Inventory Status
Using days on hand, we can now understand the inventory status for each medication at each location whether it is understocked, overstocked or at an optimal range. We can use this later as inventory optimization analyses, performance indicator for inventory management efficiency or reordering understocked items or transfering overstocked items for each location.

In [ ]:
# Flag items as 'Critical Low', 'Low', 'Optimal', 'High'
target_doh = 14  # General target of 2 weeks
weekly_data['inventory_status'] = pd.cut(weekly_data['days_on_hand'],
                                         bins=[-np.inf, 7, target_doh - 0.001, 21, np.inf],
                                         labels=['Critical Low', 'Low', 'Optimal', 'High'], right=False)

In [ ]:
status_distribution = weekly_data['inventory_status'].value_counts(normalize=True)
print("System-wide Inventory Status Distribution:")
print(status_distribution)

System-wide Inventory Status Distribution:
inventory_status
High            0.719883
Critical Low    0.265235
Optimal         0.008612
Low             0.006270
Name: proportion, dtype: float64


##### Calculate Inventory Turnover Rate
Based on the weekly usage and current inventory, we can calculate weekly inventory turnover rate to find how many times the current inventory was used during the week. If it's greater than 1, it would mean that it was used more than once and vice versa. In our case, an extremely high turnover may indicate a potential stockouts for critical medications.

We can use this together with other metrics such as days on hand to obtain a comprehensive view. For example, high turnover with low days on hand may indicate a need for restocking more frequently. On the other hand, low turnover with high days on hand may indicate overstocking.

In [ ]:
# Calculate inventory turnover rate
weekly_data['inventory_turnover'] = np.where(weekly_data['inv_qoh'] > 0,
                                             weekly_data['pick'] / weekly_data['inv_qoh'], np.nan)

In [ ]:
weekly_data.sample(60)

,week,cleaned_med_id,pick,waste,restock,inv_min,inv_max,inv_qoh,location_count,location_id,expected_usage,days_on_hand,inventory_status,inventory_turnover
167569,2023-02-27,070792,0.0,0.0,0.0,0.0,0.0,0.0,1,[Car1-35-04-03],0.00,NaN,NaN,NaN
205329,2023-05-15,057891,56.0,0.0,196.0,1190.0,2240.0,1379.0,2,"[Car4-02-11-01, Car5-12-03-01]",203.00,4.755172e+01,High,0.040609
58010,2022-07-18,075680,22.0,0.0,24.0,0.0,0.0,0.0,1,[Car4-09-12-01],12.75,0.000000e+00,Critical Low,NaN
204711,2023-05-15,028109,463.0,0.0,312.0,2100.0,2800.0,1778.0,2,"[Car2-30-08-01, CLNC-01-01-04-06]",493.50,2.521986e+01,High,0.260405
158965,2023-02-13,009117,10.0,0.0,0.0,210.0,350.0,385.0,1,[Car3-20-04-01],7.50,3.593333e+02,High,0.025974
231213,2023-07-10,51911,14.0,0.0,0.0,126.0,182.0,175.0,1,[Car3-20-12-03],8.50,1.441176e+02,High,0.080000
18873,2022-04-25,051909,0.0,0.0,0.0,28.0,84.0,42.0,1,[Car5-29-08-03],0.00,NaN,NaN,0.000000
54674,2022-07-11,075268,0.0,0.0,0.0,91.0,140.0,203.0,1,[Car3-22-12-01],0.00,NaN,NaN,0.000000
103246,2022-10-24,003707,6.0,0.0,0.0,70.0,294.0,112.0,2,"[Car1-23-12-03, Car4-22-01-03]",19.00,4.126316e+01,High,0.053571
28054,2022-05-16,048752,0.0,0.0,0.0,14.0,14.0,28.0,1,[Car3-16-06-02],0.00,NaN,NaN,0.000000


In [ ]:
weekly_data.isnull().sum()

week                       0
cleaned_med_id             0
pick                       0
waste                      0
restock                    0
inv_min                    0
inv_max                    0
inv_qoh                    0
location_count             0
location_id                0
expected_usage             0
days_on_hand          106841
inventory_status      106841
inventory_turnover    128034
dtype: int64

In [ ]:
weekly_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407137 entries, 1052 to 407136
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   week                407137 non-null  datetime64[ns]
 1   cleaned_med_id      407137 non-null  object        
 2   pick                407137 non-null  float64       
 3   waste               407137 non-null  float64       
 4   restock             407137 non-null  float64       
 5   inv_min             407137 non-null  float64       
 6   inv_max             407137 non-null  float64       
 7   inv_qoh             407137 non-null  float64       
 8   location_count      407137 non-null  int64         
 9   location_id         407137 non-null  object        
 10  expected_usage      407137 non-null  float64       
 11  days_on_hand        300296 non-null  float64       
 12  inventory_status    300296 non-null  category      
 13  inventory_turnover  279103 non-

#### References
[1] Katana. (n.d.). A guide to inventory days on hand (DOH). Retrieved July 6th, 2024, from https://katanamrp.com/inventory-days-on-hand/